# Index-Objekte

Die Index-Objekte von pandas sind für die Achsenbeschriftungen und andere Metadaten, wie den Achsennamen, verantwortlich. Jedes Array oder jede andere Sequenz von Beschriftungen, die ihr bei der Konstruktion einer Serie oder eines DataFrame verwendet, wird intern in einen Index umgewandelt:

In [1]:
import pandas as pd

obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [2]:
obj.index

DatetimeIndex(['2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05',
               '2022-02-06', '2022-02-07', '2022-02-08'],
              dtype='datetime64[ns]', freq='D')

In [3]:
obj.index[3:]

DatetimeIndex(['2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08'], dtype='datetime64[ns]', freq='D')

Indexobjekte sind unveränderlich (immutable) und können daher vom Benutzer nicht geändert werden:

In [4]:
obj.index[1] = '2022-02-03'

TypeError: Index does not support mutable operations

Die Unveränderlichkeit macht die gemeinsame Nutzung von Indexobjekten in Datenstrukturen sicherer:

In [5]:
import numpy as np

labels = pd.Index(np.arange(3))

labels

Int64Index([0, 1, 2], dtype='int64')

In [6]:
obj2 = pd.Series(np.random.randn(3),index=labels)

In [7]:
obj2

0   -0.149641
1   -0.130634
2    0.115944
dtype: float64

In [8]:
obj2.index is labels

True

Um einem Array ähnlich zu sein verhält sich ein Index auch wie eine Menge mit fester Größe:

In [9]:
data1 = {'Code': ['U+0000', 'U+0001', 'U+0002', 'U+0003', 'U+0004', 'U+0005'],
         'Decimal': [0, 1, 2, 3, 4, 5],
         'Octal': ['001', '002', '003', '004', '004', '005']}
df1 = pd.DataFrame(data1)

In [10]:
df1

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005


In [11]:
df1.columns

Index(['Code', 'Decimal', 'Octal'], dtype='object')

In [12]:
'Code' in df1.columns

True

In [13]:
'Key' in df1.columns

False

## Achsenindizes mit doppelten Labels

Anders als Python-Sets kann ein Pandas-Index doppelte Label enthalten:

In [14]:
data2 = {'Code': ['U+0006', 'U+0007'],
        'Decimal': [6, 7],
        'Octal': ['006', '007']}
df2 = pd.DataFrame(data2)
df12 = df1.append(df2)

df12

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005
0,U+0006,6,006
1,U+0007,7,007


Bei Auswahlen mit doppelten Bezeichnungen werden alle Vorkommen der betreffenden Bezeichnung ausgewählt:

In [15]:
df12.loc[1]

,Code,Decimal,Octal
1,U+0001,1,002
1,U+0007,7,007


In [16]:
df12.loc[2]

Code       U+0002
Decimal         2
Octal         003
Name: 2, dtype: object

Die Datenauswahl ist einer der Hauptpunkte, der sich bei Duplikaten anders verhält. Die Indizierung eines Labels mit mehreren Einträgen ergibt eine Serie, während einzelne Einträge einen Einzelwert ergeben. Dies kann euren Code komplizierter machen, da der Ausgabetyp der Indizierung je nachdem, ob ein Label wiederholt wird oder nicht, variieren kann. Zudem setzen viele Pandas-Funktionen, wie z.B. `reindex`, voraus, dass Label eindeutig sind. Anhand der Eigenschaft `is_unique` des Index könnt ihr feststellen, ob seine Label eindeutig sind oder nicht:

In [17]:
df12.index.is_unique

False

Um doppelte Label zu vermeiden, könnt ihr z.B. `ignore_index=True` verwenden:

In [18]:
df12 = df1.append(df2, ignore_index=True)

df12

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005
6,U+0006,6,006
7,U+0007,7,007


## Einige Indexmethoden und -eigenschaften

Jeder Index verfügt über eine Reihe von Methoden und Eigenschaften für die Mengenlogik, die andere allgemeine Fragen zu den darin enthaltenen Daten beantworten. Im folgenden einige nützliche Methoden und Eigenschaften:

Methode | Beschreibung
:------ | :-----------
`append` | verkettet zusätzliche Indexobjekte, wodurch ein neuer Index entsteht
`difference` | berechnet die Differenz zweier Mengen als Index
`intersection` | berechnet die Schnittmenge
`union` | berechnet die Vereinigungsmenge
`isin` | berechnet ein boolesches Array, das angibt, ob jeder Wert in der übergebenen Sammlung enthalten ist
`delete` | berechnet einen neuen Index, wobei das Element in Index `i` gelöscht wird
`drop` | berechnet einen neuen Index durch Löschen der übergebenen Werte
`insert` | berechnet neuen Index durch Einfügen des Elements in den Index `i`
`is_monotonic` | gibt `True` zurück, wenn jedes Element größer oder gleich dem vorherigen Element ist
`is_unique` | gibt `True` zurück, wenn der Index keine doppelten Werte enthält.
`unique` | berechnet das Array der eindeutigen Werte im Index

## Neuindizierung

Eine wichtige Methode für Pandas-Objekte ist die Neuindizierung, d.h. die Erstellung eines neuen Objekts mit neu angeordneten Werten, die mit dem neuen Index übereinstimmen. Betrachtet z.B.:

In [19]:
obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [20]:
obj

2022-02-02    0
2022-02-03    1
2022-02-04    2
2022-02-05    3
2022-02-06    4
2022-02-07    5
2022-02-08    6
Freq: D, dtype: int64

In [21]:
new_index = pd.date_range("2022-02-03", periods=7)

In [22]:
obj.reindex(new_index)

2022-02-03    1.0
2022-02-04    2.0
2022-02-05    3.0
2022-02-06    4.0
2022-02-07    5.0
2022-02-08    6.0
2022-02-09    NaN
Freq: D, dtype: float64

`reindex` erstellt einen neuen Index und indiziert den DataFrame neu. Standardmäßig werden Werte im neuen Index, für die es keine entsprechenden Datensätze im DataFrame gibt, zu `NaN`.

Bei geordneten Daten wie Zeitreihen kann es wünschenswert sein, bei der Neuindizierung Werte zu interpolieren oder zu füllen. Die Option `method` ermöglicht dies mit einer Methode wie `ffill`, die die Werte vorwärts füllt:

In [23]:
obj.reindex(new_index, method='ffill')

2022-02-03    1
2022-02-04    2
2022-02-05    3
2022-02-06    4
2022-02-07    5
2022-02-08    6
2022-02-09    6
Freq: D, dtype: int64

Bei einem DataFrame kann `reindex` entweder den (Zeilen-)Index, die Spalten oder beides ändern. Wenn nur eine Sequenz übergeben wird, werden die Zeilen im Ergebnis neu indiziert:

In [24]:
df1.reindex(range(7))

,Code,Decimal,Octal
0,U+0000,0.0,001
1,U+0001,1.0,002
2,U+0002,2.0,003
3,U+0003,3.0,004
4,U+0004,4.0,004
5,U+0005,5.0,005
6,NaN,NaN,NaN


Die Spalten können mit dem Schlüsselwort `columns` neu indiziert werden:

In [25]:
encoding = ['Octal', 'Code', 'Description']

df1.reindex(columns=encoding)

,Octal,Code,Description
0,001,U+0000,NaN
1,002,U+0001,NaN
2,003,U+0002,NaN
3,004,U+0003,NaN
4,004,U+0004,NaN
5,005,U+0005,NaN


### Argumente der Funktion `reindex`

Argument | Beschreibung
:------- | :-----------
`labels` | Neue Sequenz, die als Index verwendet werden soll. Kann eine Index-Instanz oder eine andere sequenzähnliche Python-Datenstruktur sein. Ein Index wird genau so verwendet, wie er ist, ohne dass er kopiert wird.
`axis` | Die neu zu indizierende Achse, entweder `index` (Zeilen) oder `columns` (Spalten). Die Vorgabe ist `index`. Ihr könnt alternativ `reindex(index=new_labels)` oder `reindex(columns=new_labels)` verwenden.
`method` | Interpolationsmethode; `ffill` füllt vorwärts, während `bfill` rückwärts füllt.
`fill_value` | Ersatzwert, der zu verwenden ist, wenn fehlende Daten durch Neuindizierung eingefügt werden. Verwendet `fill_value='missing'` (das Standardverhalten), wenn die fehlenden Bezeichnungen im Ergebnis Nullwerte haben sollen.
`limit` |  Beim Vorwärts- oder Rückwärtsfüllen die maximale Anzahl der zu füllenden Elemente.
`tolerance` | Beim Vorwärts- oder Rückwärtsauffüllen die maximale Größe der Lücke, die bei ungenauen Übereinstimmungen gefüllt werden soll.
`level` |  Einfachen Index auf Ebene von `MultiIndex` abgleichen; andernfalls Teilmenge auswählen.
`copy` | Wenn `True`, werden die zugrunde liegenden Daten immer kopiert, auch wenn der neue Index dem alten Index entspricht; wenn `False`, werden die Daten nicht kopiert, wenn die Indizes gleichwertig sind.

## Achsenindizes umbenennen

Die Achsenbeschriftungen können durch eine Funktion oder ein Mapping umgewandelt werden, um neue, anders beschriftete Objekte zu erzeugen. Ihr könnt auch die Achsen an Ort und Stelle ändern, ohne eine neue Datenstruktur zu erstellen. Hier ist ein einfaches Beispiel:

In [26]:
df3 = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=['Deutsch', 'English', 'Français'],
                   columns=[1,2,3,4])

df3

,1,2,3,4
Deutsch,0,1,2,3
English,4,5,6,7
Français,8,9,10,11


Wie `Series` haben auch die Achsenindizes eine `map`-Methode:

In [27]:
transform = lambda x: x[:2].upper()

df3.index.map(transform)

Index(['DE', 'EN', 'FR'], dtype='object')

Ihr könnt den Index zuweisen und den DataFrame an Ort und Stelle ändern:

In [28]:
df3.index = df3.index.map(transform)

df3

,1,2,3,4
DE,0,1,2,3
EN,4,5,6,7
FR,8,9,10,11


Wenn ihr eine umgewandelte Version eures Datensatzes erstellen möchtet ohne das Original zu verändern, könnt ihr `rename` verwenden:

In [29]:
df3.rename(index=str.lower)

,1,2,3,4
de,0,1,2,3
en,4,5,6,7
fr,8,9,10,11


Insbesondere kann `rename` in Verbindung mit einem `dict`-ähnlichen Objekt verwendet werden, das neue Werte für eine Teilmenge der Achsenbeschriftungen liefert:

In [30]:
df3.rename(index={'DE': 'BE', 'EN': 'DE', 'FR': 'EN'},
           columns={1: 0, 2: 1, 3: 2,  4: 3})

,0,1,2,3
BE,0,1,2,3
DE,4,5,6,7
EN,8,9,10,11


`rename` erspart euch das manuelle Kopieren des DataFrame und die Zuweisung seiner Index- und Spaltenattribute. Wenn ihr einen Datensatz an Ort und Stelle ändern möchtet, übergebt zusätzlich noch `inplace=True`:

In [31]:
df3.rename(index={'DE': 'BE', 'EN': 'DE', 'FR': 'EN'},
           columns={1: 0, 2: 1, 3: 2,  4: 3},
           inplace=True)

df3

,0,1,2,3
BE,0,1,2,3
DE,4,5,6,7
EN,8,9,10,11
